# Create arrow dataset

The BrainLM processing function involves doing train-validation-test split.
They hard coded to only use data with more than 200 time points, and there are a lot of the hard coded, unclear data scaling and normalisation. 
Need further investigation.

In [1]:
import numpy as np
import nibabel as nib
import os
import argparse
from math import ceil
import pickle

import pandas as pd
from datasets import Dataset, concatenate_datasets
from tqdm import tqdm
import glob
from importlib.resources import files

In [25]:
args = {
    "uk_biobank_dir": "../data/interim/brainlm_a424",     # "Path to directory containing dat files, A424 coordinates file, and A424 excel sheet.",
    "arrow_dataset_save_directory": "../data/processed/brainlm_a424",     # "The directory where you want to save the output arrow datasets."
    "dataset_name": "test_data_arrow_norm",
}
save_path = "../data/processed/brainlm_a424"

ts_min_length = 150

In [5]:
all_dat_files = os.listdir(args["uk_biobank_dir"])
all_dat_files = [filename for filename in all_dat_files if ".dat" in filename]
try:
    all_dat_files.remove("A424_Coordinates.dat")
    print('A424_Coordinates was removed from the list')
except ValueError:
    print("There's no A24 Coordinates dat file")
all_dat_files.sort()  # Sorted in ascending order, first 80% will be train. Assuming no bias in patient order


There's no A24 Coordinates dat file


In [8]:
train_split_idx = len(all_dat_files)
train_files = all_dat_files[:train_split_idx]
sh_35 = 0
sh_less = 0
for idx,file in enumerate(tqdm(all_dat_files)):
    try:
        sample = np.loadtxt(os.path.join(args["uk_biobank_dir"],file)).T #490, 424
        if sample.shape[0] < ts_min_length:
            print(sample.shape, idx, "ommitted due to insufficient data")
            sh_less += 1
        else:
            sh_35 += 1
        # print(sample.shape)
    except UnicodeDecodeError:
        print(file)


100%|██████████| 155/155 [00:06<00:00, 23.23it/s]


In [9]:
compute_Stats=True
if compute_Stats:
    num_files = sh_35 #len(all_dat_files_rs) + len(all_dat_files_tf)
    all_stds = np.zeros([num_files, 424])
    all_data = np.empty([num_files*ts_min_length, 424])
    for idx,file in enumerate(tqdm(train_files)):
        if idx == num_files:
            break
        # if idx%2000==0:
        #     print('idx: {}, next file: {}'.format(idx,file))
        try:
            sample = np.loadtxt(os.path.join(args["uk_biobank_dir"],file)) #490, 424
            # print(sample.shape)
        except UnicodeDecodeError:
            print(file)
        # sample = np.loadtxt(os.path.join(uk_biobank_dir_rs, file, 'rfMRI_REST','rfMRI_REST_Atlas_MSMAll_hp2000_clean_MGTR_zscored_HCP_MMP_BNAC.dat')).astype(np.float32).T
        sample_mean = sample.mean(axis=0, keepdims=True)
        sample_mean = sample_mean[None,:].repeat(sample.shape[0],1).squeeze()
        sample = sample - sample_mean

        idx_sample=idx


        if sample.shape[0] < ts_min_length:
            continue
        try:
            all_data[idx*ts_min_length:(idx+1)*ts_min_length,:] = sample[:ts_min_length,:]
        except ValueError:
            print(sample.shape)
            print('idx: {}, idx_sample: {}'.format(idx,idx_sample))

    global_std = np.std(all_data, axis=0)
    data_median_per_voxel = np.median(all_data,axis=0)
    data_mean_per_voxel = np.mean(all_data,axis=0)

    all_data_nonzeros = np.copy(all_data)
    all_data_nonzeros[all_data_nonzeros == 0] = 'nan'
    quartiles = np.nanpercentile(all_data_nonzeros, [25, 75], axis=0)
    IQR = quartiles[1,:]-quartiles[0,:]

100%|██████████| 155/155 [00:06<00:00, 23.60it/s]
/home/hwang/simexp/hwang/hfplayground/.venv/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:1620: RuntimeWarning: All-NaN slice encountered
  return fnb._ureduce(a,


In [23]:
# --- Normalization Calculations ---#
# Calculate min and max value across train, validation, and test sets
global_train_max = -1e9
global_train_min = 1e9
voxel_maximums_train = []
voxel_minimums_train = []

for filename in tqdm(train_files, desc="Getting normalization stats"):
    dat_arr = np.loadtxt(os.path.join(args["uk_biobank_dir"], filename)).astype(
        np.float32
    ).T
    #assert (
    #    np.min(dat_arr) >= 0
    #), "Minimum of patient recording is a negative number, check normalization"
    if np.max(dat_arr) > global_train_max:
        global_train_max = np.max(dat_arr)
    if np.min(dat_arr) < global_train_min:
        global_train_min = np.min(dat_arr)

    dat_arr_max = np.max(dat_arr, axis=1)
    dat_arr_min = np.min(dat_arr, axis=1)
    voxel_maximums_train.append(dat_arr_max)
    voxel_minimums_train.append(dat_arr_min)

voxel_maximums_train = np.stack(voxel_maximums_train, axis=0)  # stack in time dimension
voxel_minimums_train = np.stack(voxel_minimums_train, axis=0)
global_per_voxel_train_max = np.max(voxel_maximums_train, axis=0)
global_per_voxel_train_min = np.min(voxel_minimums_train, axis=0)

# --- Convert All .dat Files to Arrow Datasets ---#
# Training set
train_dataset_dict = {
    "Raw_Recording": [],
    "Voxelwise_RobustScaler_Normalized_Recording": [],
    "All_Patient_All_Voxel_Normalized_Recording": [],
    "Per_Patient_All_Voxel_Normalized_Recording": [],
    "Per_Patient_Per_Voxel_Normalized_Recording": [],
    "Per_Voxel_All_Patient_Normalized_Recording": [],
    "Subtract_Mean_Normalized_Recording": [],
    "Subtract_Mean_Divide_Global_STD_Normalized_Recording": [],
    "Subtract_Mean_Divide_Global_99thPercent_Normalized_Recording": [],
    "Filename": [],
    "Patient ID": [],
}

Getting normalization stats: 100%|██████████| 155/155 [00:06<00:00, 23.85it/s]


In [26]:
for filename in tqdm(train_files, desc="Normalizing Data"):
    dat_arr = np.loadtxt(os.path.join(args["uk_biobank_dir"], filename)).astype(
        np.float32
    ).T

    if dat_arr.shape[0] < ts_min_length:
        continue

    if dat_arr.shape[0] > 424:
        dat_arr = dat_arr[:350, :]

    global_norm_dat_arr = np.copy(dat_arr)
    per_patient_all_voxels_norm_dat_arr = np.copy(dat_arr)
    per_patient_per_voxel_norm_dat_arr = np.copy(dat_arr)
    per_voxel_all_patient_norm_dat_arr = np.copy(dat_arr)
    recording_mean_subtracted = np.copy(dat_arr)
    recording_mean_subtracted2 = np.copy(dat_arr)
    recording_mean_subtracted3 = np.copy(dat_arr.T)
    global_std = 41.44047  # calculated in normalization notebook
    _99th_percentile = 111.13143061224855  # calculated externally

    # All patients, all voxels normalization
    if (global_train_max - global_train_min) > 0.0:
        global_norm_dat_arr = (global_norm_dat_arr - global_train_min) / (
            global_train_max - global_train_min
        )

    # Per patient all voxel normalization
    patient_all_voxel_min_val = np.min(per_patient_all_voxels_norm_dat_arr)
    patient_all_voxel_max_val = np.max(per_patient_all_voxels_norm_dat_arr)
    if (patient_all_voxel_max_val - patient_all_voxel_min_val) > 0.0:
        per_patient_all_voxels_norm_dat_arr = (
            per_patient_all_voxels_norm_dat_arr - patient_all_voxel_min_val
        ) / (patient_all_voxel_max_val - patient_all_voxel_min_val)

    # Per patient per voxel normalization
    for voxel_idx in range(dat_arr.shape[1]):
        patient_voxel_min_val = per_patient_per_voxel_norm_dat_arr[
            :, voxel_idx
        ].min()
        patient_voxel_max_val = per_patient_per_voxel_norm_dat_arr[
            :, voxel_idx
        ].max()
        if (patient_voxel_max_val - patient_voxel_min_val) > 0.0:
            per_patient_per_voxel_norm_dat_arr[:, voxel_idx] = (
                per_patient_per_voxel_norm_dat_arr[:, voxel_idx]
                - patient_voxel_min_val
            ) / (patient_voxel_max_val - patient_voxel_min_val)

    # Per voxel all patient normalization
    for voxel_idx in range(dat_arr.shape[1]):
        voxel_maximum = global_per_voxel_train_max[voxel_idx]
        voxel_minimum = global_per_voxel_train_min[voxel_idx]
        if (voxel_maximum - voxel_minimum) > 0.0:
            per_voxel_all_patient_norm_dat_arr[:, voxel_idx] = (
                per_voxel_all_patient_norm_dat_arr[:, voxel_idx] - voxel_minimum
            ) / (voxel_maximum - voxel_minimum)

    # Subtract Mean, Scale by Global Standard Deviation normalization
    for voxel_idx in range(dat_arr.shape[1]):
        voxel_mean = recording_mean_subtracted[:, voxel_idx].mean()
        recording_mean_subtracted[:, voxel_idx] = (
            recording_mean_subtracted[:, voxel_idx] - voxel_mean
        )

    z_score_global_recording = np.divide(recording_mean_subtracted, global_std)

    # Subtract Mean, Scale by global 99th percentile
    for voxel_idx in range(dat_arr.shape[1]):
        voxel_mean = recording_mean_subtracted2[:, voxel_idx].mean()
        recording_mean_subtracted2[:, voxel_idx] = (
            recording_mean_subtracted2[:, voxel_idx] - voxel_mean
        )

    #Voxelwise Robust Scaler Normalization
    recording_mean_subtracted3 = recording_mean_subtracted3 - recording_mean_subtracted3.mean(axis=0)
    recording_mean_subtracted3 = (recording_mean_subtracted3 - data_median_per_voxel / IQR)

    _99th_global_recording = np.divide(recording_mean_subtracted2, _99th_percentile)

    train_dataset_dict["Raw_Recording"].append(dat_arr)
    train_dataset_dict["Voxelwise_RobustScaler_Normalized_Recording"].append(recording_mean_subtracted3)
    train_dataset_dict["All_Patient_All_Voxel_Normalized_Recording"].append(
        global_norm_dat_arr
    )
    train_dataset_dict["Per_Patient_All_Voxel_Normalized_Recording"].append(
        per_patient_all_voxels_norm_dat_arr
    )
    train_dataset_dict["Per_Patient_Per_Voxel_Normalized_Recording"].append(
        per_patient_per_voxel_norm_dat_arr
    )
    train_dataset_dict["Per_Voxel_All_Patient_Normalized_Recording"].append(
        per_voxel_all_patient_norm_dat_arr
    )
    train_dataset_dict["Subtract_Mean_Normalized_Recording"].append(
        recording_mean_subtracted
    )
    train_dataset_dict[
        "Subtract_Mean_Divide_Global_STD_Normalized_Recording"
    ].append(z_score_global_recording)
    train_dataset_dict[
        "Subtract_Mean_Divide_Global_99thPercent_Normalized_Recording"
    ].append(_99th_global_recording)
    train_dataset_dict["Filename"].append(filename)
    train_dataset_dict["Patient ID"].append(filename.split(".dat")[-1])

arrow_train_dataset = Dataset.from_dict(train_dataset_dict)
arrow_train_dataset.save_to_disk(
    dataset_path=os.path.join(save_path, "train")
)

Normalizing Data: 100%|██████████| 155/155 [00:08<00:00, 17.62it/s]


Saving the dataset (0/2 shards):   0%|          | 0/310 [00:00<?, ? examples/s]

In [27]:
# --- Save Brain Region Coordinates Into Another Arrow Dataset ---#
coords_dat = np.loadtxt(files('hfplayground') / "data/brainlm/atlases/A424_Coordinates.dat").astype(np.float32)
coords_pd = pd.DataFrame(coords_dat, columns=["Index", "X", "Y", "Z"])
coords_dataset = Dataset.from_pandas(coords_pd)
coords_dataset.save_to_disk(
    dataset_path=os.path.join(save_path, "Brain_Region_Coordinates")
)
print("Done.")

Saving the dataset (0/1 shards):   0%|          | 0/424 [00:00<?, ? examples/s]

Done.


In [28]:
coords_pd

,Index,X,Y,Z
0,1.0,14.0,-78.0,4.0
1,2.0,44.0,-64.0,4.0
2,3.0,18.0,-76.0,32.0
3,4.0,10.0,-74.0,0.0
4,5.0,18.0,-92.0,16.0
...,...,...,...,...
419,420.0,0.0,-56.0,-38.0
420,421.0,6.0,-52.0,-48.0
421,422.0,-20.0,-36.0,-44.0
422,423.0,0.0,-48.0,-36.0
